<a href="https://colab.research.google.com/github/jacobmrCuzzins/Machine-DeepLearning-/blob/main/CNN_model_LeNet_structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a Model based on the LeNet layer structure

The below notebook shows my implementation of the LeNet structure to see how efficient it may be on image classification when compared to a basic CNN or MLP model

## Accessing Data from Google Drive
The dataset for this assignment is the CIFAR-10 dataset that can be found here:
https://www.cs.toronto.edu/~kriz/cifar.html

The CIFAR-10 and CIFAR-100 are well studied, yet challenging image recognition dataset. The CIFAR-10 has up to 10 classes to classify and contains 60,000 32x32 images. You should read the description of the dataset and download the dataset for Python, that is

CIFAR-10 python version: https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

Once downloaded you need to then extract and upload the `cifar-10-batches-py` directory your Google Drive so that you can access it from within your Google Colab.

You can mount the Google Drive from the menu on the left or uncomment use the code below mount the drive.  See here for documentation on file access in Colab:

[External data: Local Files, Drive, Sheets, and Cloud Storage](https://colab.research.google.com/notebooks/io.ipynb)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Functions to work with CIFAR

The functions below help with access to the CIFAR-10 data the you have downloaded.

In [ ]:
import pickle
import numpy as np
import pandas as pd
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

def load_CIFAR_batch(filename, flatten=True, categorical=True):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding='bytes')
        X = datadict[b'data']
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        if (flatten):
          X = X.reshape(10000, 3072)
        X = X.astype('float32')
        X /= 255

        y = datadict[b'labels']
        y = np.array(y)
        if (categorical):
          y = pd.get_dummies(y).values

        return X, y

def load_CIFAR_meta(filename):
  with open(filename,'rb') as f:
    metadict = pickle.load(f, encoding='bytes')

    class_labels = [ val.decode() for val in metadict.get(b'label_names') ]
    return class_labels

def get_image(X, index, nchans=3, size=32):
  xi = X[index,:]
  img = xi.reshape(32, 32, 3)
  return img

## Load the CIFAR data

The CIFAR data has 5 batches of data and 1 test data set. Each batch is labelled
- `data_batch_1`
- `data_batch_2`
- `data_batch_3`
- `data_batch_4`
- `data_batch_5`

and a test set labelled
- `test_batch`

each batch has 10,000 images, so 50,000 training and 10,000 test images.

Below is example of loading the first batch of training data labelled as `data_batch_1`.  You will need to update the path to match where you have stored your cifar-10 data.

LOADING ONLY ONE BATCH

In [ ]:
X, y = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_1', flatten = False)

# split into train and test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y)

# Create 1d version for testing accuracy
ytrain_1d = np.argmax(ytrain, axis=1)
ytest_1d = np.argmax(ytest, axis=1)

print('We have {} instances of images'.format(y.shape[0]))

We have 10000 instances of images


TO LOAD THE FULL BATCH

In [ ]:
flatten = False
categorical = True

# load only the first batch
#X1, y1 = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_1',flatten=flatten,categorical=categorical)

# load only the second batch
#X2, y2 = load_CIFAR_batch('/content/drive/My Drive/COMP2712/data/cifar-10-batches-py/data_batch_2',flatten=flatten,categorical=categorical)

# Load the first batch from CIFAR-10
X, y = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_1',flatten= False ,categorical=categorical)

# iterate over 2 to 5
for bi in range(2,6):
  # load the next data set 'bi'
  X2, y2 = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_{}'.format(bi),flatten=flatten,categorical=categorical)

  # concatenate/stack the dataest together
  X = np.vstack([X, X2])
  y = np.vstack([y, y2])

 #Load the test data into test variables
Xtest, ytest = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/test_batch', flatten=False)

# Create 1d version for testing accuracy
ytrain_1d = np.argmax(y, axis=1)
ytest_1d = np.argmax(ytest, axis=1)

print('We have {} instances of images'.format(y.shape[0]))


We have 50000 instances of images


The number of instances/examples for all the different classes.  There are 10 different classes.

In [ ]:
[np.sum(np.argmax(y, axis=1) == i) for i in range(0,10)]

[5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]

The labels for the classes are stored in the `batches.meta` file

In [ ]:
class_labels = load_CIFAR_meta('/content/drive/My Drive/cifar-10-batches-py/batches.meta')
print(class_labels)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


-

In [ ]:
# sklearn evaluation packages
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

**BUILD THE CNN MODEL FROM ASSIGNMENT SHEET**

This is the CNN Model built from the given structure in the assignment

In [ ]:
X_flatten = X.reshape((50000, -1))

**LeNet Implementation**

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, AveragePooling2D, Dropout

model_lenet = Sequential()

# Layer 1: Convolutional Layer
model_lenet.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 3), padding='same'))

# Layer 2: Average Pooling
model_lenet.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 3: Convolutional Layer
model_lenet.add(Conv2D(16, kernel_size=(5, 5), activation='relu', padding='valid'))

# Layer 4: Average Pooling
model_lenet.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 5: Flatten (Flatten)
model_lenet.add(Flatten())

# Layer 6: Fully Connected Layer
model_lenet.add(Dense(120, activation='relu'))

# Layer 8: Fully Connected Layer
model_lenet.add(Dense(84, activation='relu'))

# Layer 9: Output Layer
model_lenet.add(Dense(10, activation='softmax'))

model_lenet.compile(optimizer=tf.optimizers.Adam(),
              loss=keras.losses.categorical_crossentropy,
              metrics=keras.metrics.categorical_crossentropy)

model_lenet.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 6)         456       
                                                                 
 average_pooling2d (Average  (None, 16, 16, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 16)        2416      
                                                                 
 average_pooling2d_1 (Avera  (None, 6, 6, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten (Flatten)           (None, 576)               0         
                                                                 
 dense (Dense)               (None, 120)               6

**Stratified K-Fold Evaluation of the model**

In [ ]:
# sklearn evaluation packages
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
# Stratified K-Fold for evaluation of generalisation performance
k = 5 # 10 is the gold standard
kf = StratifiedKFold(n_splits=k)
foldi = 1
ac = [] # accuracy_score
cr = [] # classification_report
cm = [] # confusion_matrix
y_test_max_all = []
y_pred_max_all = []

activation = 'relu'
no_epochs = 5


for train_index, test_index in kf.split(X_flatten,ytrain_1d):

  print('Training with {0} for {1} for fold {2} or {3}'.format(activation, no_epochs, foldi, k))
  history = model_lenet.fit(X, y, epochs=no_epochs, verbose=1)

  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  loss_final = np.sqrt(float(hist['loss'].tail(1)))
  print()
  print('Final Loss on training set: {}'.format(round(loss_final, 3)))

  # evaluate the model
  y_pred_real = model_lenet.predict(Xtest)
  y_pred_max = np.argmax(y_pred_real, axis=-1).astype(int)
  y_test_max = ytest.argmax(axis=1).astype(int)

  y_pred_max_all.extend(y_pred_max)
  y_test_max_all.extend(y_test_max)

  ac.append(accuracy_score(y_test_max,y_pred_max))
  print('accuracy is {:.2f}%'.format(ac[-1]*100)) # Print accuracy score
  print()
  cr.append(classification_report(y_test_max,y_pred_max)) # Print summary report
  cm.append(confusion_matrix(y_test_max, y_pred_max))

  foldi = foldi + 1

# Calculate the average accuracy and standard deviation
average_accuracy = np.mean(ac) * 100
std_dev_accuracy = np.std(ac) * 100

# Print the results
print(f'Average Accuracy: {average_accuracy:.2f}%')
print(f'Standard Deviation of Accuracy: {std_dev_accuracy:.2f}%')

print('k-fold complete!')


Training with relu for 5 for fold 1 or 5
Epoch 1/5
1563/1563 [==============================] - 22s 6ms/step - loss: 1.6573 - categorical_crossentropy: 1.6573
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.3629 - categorical_crossentropy: 1.3629
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2560 - categorical_crossentropy: 1.2560
Epoch 4/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1828 - categorical_crossentropy: 1.1828
Epoch 5/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1171 - categorical_crossentropy: 1.1171

Final Loss on training set: 1.057
313/313 [==============================] - 1s 2ms/step
accuracy is 57.79%

Training with relu for 5 for fold 2 or 5
Epoch 1/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0595 - categorical_crossentropy: 1.0595
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.0107 - categorical_crossentr

**K-Fold Evaluation**

In [ ]:
print('Average Accuracy for {}-CV: {}% +/- {}% SD'.format(k, np.round(np.mean(ac)*100,2), np.round(np.std(ac)*100,2)))

Average Accuracy for 5-CV: 60.63% +/- 1.42% SD


**Accuracy Report**

In [ ]:
acc_all = accuracy_score(y_test_max_all,y_pred_max_all)*100
print('Accuracy: {:.2f}%'.format(acc_all))
print()
print(classification_report(y_test_max_all,y_pred_max_all))
print(confusion_matrix(y_test_max_all, y_pred_max_all))

mcc_all = matthews_corrcoef(y_test_max_all,y_pred_max_all)
print()
print('Matthews Correlation Coefficient: {}'.format(mcc_all))

Accuracy: 60.63%

              precision    recall  f1-score   support

           0       0.62      0.68      0.65      5000
           1       0.69      0.73      0.71      5000
           2       0.52      0.49      0.50      5000
           3       0.42      0.42      0.42      5000
           4       0.55      0.52      0.54      5000
           5       0.54      0.45      0.49      5000
           6       0.65      0.71      0.68      5000
           7       0.69      0.66      0.68      5000
           8       0.69      0.73      0.71      5000
           9       0.65      0.65      0.65      5000

    accuracy                           0.61     50000
   macro avg       0.60      0.61      0.60     50000
weighted avg       0.60      0.61      0.60     50000

[[3406  176  222   91  129   22   74   76  565  239]
 [ 162 3672   53   79   20   23   69   45  247  630]
 [ 475   67 2435  398  517  289  391  179  143  106]
 [ 157   85  408 2098  347  894  472  256  114  169]
 [ 215   65